In [1]:
from IPython.display import display
from IPython.display import HTML
import IPython.core.display as di # Example: di.display_html('<h3>%s:</h3>' % str, raw=True)

# This line will hide code by default when the notebook is exported as HTML
di.display_html('<script>jQuery(function() {if (jQuery("body.notebook_app").length == 0) { jQuery(".input_area").toggle(); jQuery(".prompt").toggle();}});</script>', raw=True)

# This line will add a button to toggle visibility of code blocks, for use with the HTML export version
di.display_html('''<button onclick="jQuery('.input_area').toggle(); jQuery('.prompt').toggle();">Toggle code</button>''', raw=True)

Toggle code

In [100]:
import os
from pathlib import Path
import glob
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
#import fun_eda

In [3]:
def abre_archivos_de_rutas():
    """"This function open a list of files and append them all together in one df"""
    
    list_files = ['rutas_0hr.csv', 'rutas_8hr.csv', 'rutas_16hr.csv']
    
    list_df = []
    for file in list_files:
        # Open csv file
        df = pd.read_csv(os.path.join('~/bj/data/rutas_tiles', file), sep=',', low_memory=False)
        
        # Add an element to the list for each activity
        list_df.append(df)
        
    # Join all activities in one df
    data = pd.concat(list_df, axis=0, ignore_index=True)
    
    return data

In [33]:
##### Load dictionario tiles - ageb
DICT_TILES_AGEB = "~/bj/data/rutas_tiles/id_tiles_ageb.csv"
dict_tiles_ageb = pd.read_csv(DICT_TILES_AGEB)
dict_tiles_ageb['coord'] = dict_tiles_ageb['long'].astype(str) + ' ' + dict_tiles_ageb['lat'].astype(str)
dict_tiles_ageb['ageb'] = dict_tiles_ageb['CVE_AGEB'].astype('str').str.zfill(4)
dict_ageb = dict(zip(dict_tiles_ageb.coord, dict_tiles_ageb.ageb))

##### Load rutas en tiles
rutas_tiles = abre_archivos_de_rutas()
rutas_tiles.rename(columns={'Unnamed: 0':'geometry'}, inplace=True)
# Separamos las coordenadas
rutas_tiles[['origen', 'destino']] = rutas_tiles['geometry'].astype(str).str.replace("(", "").str.replace(")","").str.strip().str.split(',', 2, expand=True)
rutas_tiles[['origen_long', 'origen_lat']] = rutas_tiles['origen'].str.strip().str.split(' ', 2, expand=True).astype('float64').round(8).astype('str')
rutas_tiles[['destino_long', 'destino_lat']] = rutas_tiles['destino'].str.strip().str.split(' ', 2, expand=True).astype('float64').round(8).astype('str')
rutas_tiles['coord_origen'] = rutas_tiles['origen_long'] + ' ' + rutas_tiles['origen_lat']
rutas_tiles['coord_destino'] = rutas_tiles['destino_long'] + ' ' + rutas_tiles['destino_lat']
rutas_tiles.drop(columns=['origen', 'destino','origen_long', 'origen_lat', 'destino_long', 'destino_lat'], inplace=True)
print("Rutas tiles:", rutas_tiles.shape)
#rutas_tiles.head(5)

##### Load datos poblacion por tiles
DATOS_POB_TILES = "~/bj/data/rutas_tiles/poblacion_tile.csv"
tiles_data = pd.read_csv(DATOS_POB_TILES) 
print("Info por tiles:", tiles_data.shape)
#tiles_data.head(5)

##### Load datos demograficos por AGEB
DATOS_DEM_AGEB = "~/bj/data/rutas_tiles/demograficos_agebs.csv"
ageb_data = pd.read_csv(DATOS_DEM_AGEB) 
ageb_data['coord'] = ageb_data['long'].astype(str) + ' ' + ageb_data['lat'].astype(str)
ageb_data['ageb'] = ageb_data['CVE_AGEB'].astype('str').str.zfill(4)
print("Info por AGEB:", ageb_data.shape)
#ageb_data.head(5)


Rutas tiles: (103, 73)
Info por tiles: (605, 2)
Info por AGEB: (600, 196)


In [34]:
## Join info por AGEB con pob por tile
ageb_data = ageb_data.merge(tiles_data, on='id', how='left')
print(data.shape)

(600, 197)


In [79]:
### join info por AGEB con las rutas, por su Origen
df_origen = rutas_tiles.merge(ageb_data, left_on='coord_origen', right_on = 'coord', how='left')
df_origen = df_origen.assign(punto='origen')
df_origen.drop(columns = ['coord_origen', 'coord_destino'], inplace=True)

df_destino = rutas_tiles.merge(ageb_data, left_on='coord_destino', right_on = 'coord', how='left')
df_destino = df_destino.assign(punto='destino')
df_destino.drop(columns = ['coord_origen', 'coord_destino'], inplace=True)

In [82]:
rutas_tiles_info = pd.concat([df_origen, df_destino], ignore_index=True)
print(rutas_tiles_info.shape)
rutas_tiles_info.head(3)

(206, 269)


,geometry,2020-03-30,2020-03-31,2020-04-01,2020-04-02,2020-04-03,2020-04-04,2020-04-05,2020-04-06,2020-04-07,...,VPH_LAVAD,VPH_AUTOM,VPH_PC,VPH_TELEF,VPH_CEL,VPH_INTER,coord,ageb,pob_tile,punto
0,"(-99.129638671875 19.508019823370297, -99.1296...",2.293578,-7.436570,-5.869406,2.387449,-4.501385,0.296296,-0.791139,-12.614679,-7.874016,...,2280.0,1355.0,1388.0,2129.0,2162.0,1054.0,-99.12963867 19.50801982,0718,20222.136603,origen
1,"(-99.129638671875 19.508019823370297, -99.1516...",-13.392857,1.602564,-4.419192,-1.177856,-6.008011,-10.256410,2.890173,-2.232143,-4.807692,...,2280.0,1355.0,1388.0,2129.0,2162.0,1054.0,-99.12963867 19.50801982,0718,20222.136603,origen
2,"(-99.15161132812499 19.42515338859868, -99.173...",-2.070393,-13.333333,-5.863956,-4.988488,-12.249443,-11.571842,-4.985755,-15.527950,-11.904762,...,3546.0,2020.0,3038.0,3634.0,3938.0,2550.0,-99.15161133 19.42515339,0697,7954.322222,origen


In [91]:
# Dejamos solo las variables demograficas
cols = rutas_tiles_info.columns
## Solo variables demograficas
cols = cols[(~cols.str.contains('2020')) & (~cols.str.contains('long')) & (~cols.str.contains('lat')) &
            (~cols.str.contains('id')) & (~cols.str.contains('CVE_AGEB')) & (~cols.str.contains('geometry'))]

rutas_tiles_info = rutas_tiles_info[cols]
rutas_tiles_info['coord'] = rutas_tiles_info['coord'].astype('category')
rutas_tiles_info['ageb'] = rutas_tiles_info['ageb'].astype('category')
rutas_tiles_info['punto'] = rutas_tiles_info['punto'].astype('category')

In [93]:
pd.set_option('display.max_rows', 500)
rutas_tiles_info.dtypes

POBTOT         float64
POBMAS         float64
POBFEM         float64
P_0A2          float64
P_0A2_M        float64
P_0A2_F        float64
P_3YMAS        float64
P_3YMAS_M      float64
P_3YMAS_F      float64
P_5YMAS        float64
P_5YMAS_M      float64
P_5YMAS_F      float64
P_12YMAS       float64
P_12YMAS_M     float64
P_12YMAS_F     float64
P_15YMAS       float64
P_15YMAS_M     float64
P_15YMAS_F     float64
P_18YMAS       float64
P_18YMAS_M     float64
P_18YMAS_F     float64
P_3A5          float64
P_3A5_M        float64
P_3A5_F        float64
P_6A11         float64
P_6A11_M       float64
P_6A11_F       float64
P_8A14         float64
P_8A14_M       float64
P_8A14_F       float64
P_12A14        float64
P_12A14_M      float64
P_12A14_F      float64
P_15A17        float64
P_15A17_M      float64
P_15A17_F      float64
P_18A24        float64
P_18A24_M      float64
P_18A24_F      float64
P_15A49_F      float64
P_60YMAS       float64
P_60YMAS_M     float64
P_60YMAS_F     float64
REL_H_M    

In [102]:
#num_vars = rutas_tiles_info.select_dtypes(include='number').columns.values
#fun_eda.descriptive_stats_for_numeric_vars(rutas_tiles_info, num_vars)

,POBTOT,POBMAS,POBFEM,P_0A2,P_0A2_M,P_0A2_F,P_3YMAS,P_3YMAS_M,P_3YMAS_F,P_5YMAS,...,VPH_RADIO,VPH_TV,VPH_REFRI,VPH_LAVAD,VPH_AUTOM,VPH_PC,VPH_TELEF,VPH_CEL,VPH_INTER,pob_tile
n_observations,206.000000,206.000000,206.000000,206.000000,206.000000,206.000000,206.000000,206.000000,206.000000,206.000000,...,206.000000,206.000000,206.000000,206.000000,206.000000,206.000000,206.000000,206.000000,206.000000,206.000000
max,68416.000000,33008.000000,35387.000000,3112.000000,1546.000000,1403.000000,64713.000000,31086.000000,33625.000000,62405.000000,...,14713.000000,16128.000000,14489.000000,12796.000000,8189.000000,7364.000000,10654.000000,12063.000000,5791.000000,44297.269610
min,1476.000000,684.000000,785.000000,40.000000,20.000000,20.000000,1351.000000,624.000000,727.000000,1306.000000,...,398.000000,422.000000,402.000000,330.000000,134.000000,218.000000,322.000000,320.000000,180.000000,5472.139689
mean,19430.203883,9264.849515,10153.359223,826.126214,388.257282,386.063107,18167.941748,8639.771845,9527.572816,17536.398058,...,4908.796117,5315.310680,4820.815534,4112.262136,2568.218447,2653.970874,3688.053398,4180.140777,2151.150485,20334.463467
std_dev,10855.708006,5208.901388,5646.288718,553.076208,276.183020,254.292670,10196.897653,4873.472935,5329.984886,9790.457093,...,2571.071351,2798.466036,2527.915741,2098.791129,1352.383073,1381.894641,1843.712909,2165.785023,1144.401082,9264.129214
25%,14230.000000,6646.000000,7584.000000,379.000000,177.000000,185.000000,13156.000000,5897.000000,7044.000000,12852.000000,...,3840.000000,4132.000000,3592.000000,3251.000000,1838.000000,1589.000000,2763.000000,3051.000000,1213.000000,13143.322038
median,15828.000000,7620.000000,8197.000000,702.000000,291.000000,340.000000,15038.000000,7254.000000,7783.000000,14478.000000,...,4212.000000,4500.000000,4300.000000,3546.000000,2116.000000,2317.000000,3634.000000,3938.000000,2125.000000,20222.136603
75%,26914.000000,13260.000000,14051.000000,1312.000000,608.000000,606.000000,25424.000000,12524.000000,13301.000000,24450.000000,...,7377.750000,8031.250000,7418.250000,6024.000000,3966.000000,4207.500000,5103.000000,6459.250000,3185.000000,27432.456306
kurtosis,0.877593,0.987773,0.778126,0.562112,0.626956,0.628197,0.975792,1.085482,0.872654,1.005720,...,-0.180653,-0.128431,-0.213877,0.238998,0.123358,-0.809961,-0.214946,-0.344897,-0.872894,-0.797441
skewness,0.478740,0.488350,0.472488,0.659040,0.756134,0.684887,0.480421,0.483508,0.480548,0.478813,...,0.252530,0.258341,0.301032,0.361300,0.524675,0.261209,0.295128,0.273993,0.342686,0.469888


### Escalamiento 

In [94]:
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler

In [103]:
# scale numeric variables
scaler = StandardScaler()
scaled_features = scaler.fit_transform(rutas_tiles_info[num_vars])

### DBScan

In [39]:
from sklearn.cluster import DBSCAN
from sklearn.datasets import make_moons
from sklearn.metrics import adjusted_rand_score